# Annotating ground truth for object detection

The goal of this notebook is to annotate images which can later be used for training object detection algorithms.

We will configure and run the annotation project in Toloka from scratch.

Follow the steps in this notebook to get the labeled data at the end.


## Prerequisites
To proceed with this notebook:
- Make sure you are [registered](https://toloka.ai/get-started/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-1) in Toloka as a requester.
- Use the promo code **OBJECT_DETECTION_PROMO** to add $10 to your account on your [profile page](https://platform.toloka.ai/requester/profile/income/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-2). You can use it to pay for data labeling while following this tutorial.


## The challenge
We have a set of photos with people in them:

<table  align="center">
  <tr><td>
    <img src="https://tlk.s3.yandex.net/dataset/faces_detection/5fc9583cbda646c7de9c6451b7f22a165b6b7d4b.jpg"
         alt="Sample photo of people"  width="650">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> Sample photo of people
  </td></tr>
</table>

We need to outline every person's face. Ultimately, we need to get a set of pixelwise boundaries that represent the people's faces in each photo. Here’s what it can look like:

<table  align="center">
  <tr><td>
    <img src="./img/segmentation_example.png"
         alt="Example of how people face detection can be performed"  width="650">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> An example of how face detection can be performed
  </td></tr>
</table>

This type of annotation often uses polygons. We are using bounding boxes to simplify the task so that we can reduce costs and speed things up.  You can still use polygons, just uncomment the related pieces of code.

### Detailed task description
In this notebook, we will implement projects #2 and #3 from [this Toloka tutorial](https://toloka.ai/docs/guide/concepts/image-segmentation-overview.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-3). Feel free to check them out if you want to learn how to configure these projects in the web interface. Otherwise you can skip it, since this notebook covers all the steps using the Toloka API.

We’ll skip the first project “Does the image contain a specific object?” from the tutorial above because it’s easy to implement using our “verification project” code.

We will implement two projects in Toloka:
- A detection project called “[Select an object in the image](https://toloka.ai/docs/guide/concepts/image-segmentation-project2.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-4)”: Tolokers will select image areas that contain faces. Tolokers are people around the world who get paid for completing your tasks. 
- A verification project called “[Are the bounding boxes correct?](https://toloka.ai/docs/guide/concepts/image-segmentation-project3.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-5)”: Tolokers will check the annotated images.

We don’t use [control tasks](https://toloka.ai/docs/guide/concepts/goldenset.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-6) or [majority vote](https://toloka.ai/docs/guide/concepts/mvote.html?utm_source=github&utm_medium=instruction-b&utm_campaign=link-7) to control quality in the detection project because we can’t expect the image annotations provided by the Tolokers to match each other exactly to the pixel. Instead, we’ll check detection results in the second project, where a different group of Tolokers will decide whether the faces are annotated correctly or not.

### Set up the environment

Prepare the environment and import necessary libraries.

In [ ]:
!pip install toloka-kit==1.0.0 # To interact with Toloka API
!pip install ipyplot # To plot images inside Jupyter Notebooks cells
!pip install crowd-kit==1.1.0

import os
import datetime
import time
import logging
import sys

import pandas as pd   # To perform data manipulation
import ipyplot


from typing import List
from toloka.streaming.event import AssignmentEvent

import toloka.client as toloka
import toloka.client.project.template_builder as tb

from crowdkit.aggregation import MajorityVote

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

Create a toloka-client instance. All API calls will pass through it.

To get an OAuth token, follow these [instructions](https://toloka.ai/en/docs/api/concepts/access/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-8). Enter your token.

In [ ]:
toloka_client = toloka.TolokaClient(input("Enter your token:"), 'PRODUCTION')
print(toloka_client.get_requester())

Check out Toloka documentation to learn more about [the Toloka API](https://toloka.ai/en/docs/api/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-9) and [Toloka-Kit](https://toloka.ai/en/docs/toloka-kit/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-10).

### Review the dataset
We use the [Human Parts Dataset](https://github.com/xiaojie1017/Human-Parts/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-11), distributed under the MIT license.

Our dataset is just a collection of URLs leading to JPEG images. GIF, PNG, and WEBP formats are also supported.


In [ ]:
!curl https://tlk.s3.yandex.net/dataset/faces_detection.tsv --output dataset.tsv # TODO

# Load the dataset of links to a pd DataFrame
dataset = pd.read_csv('dataset.tsv', sep='\t')

# Plot 5 images from dataset to verify data loading
ipyplot.plot_images(
    [url for url in dataset['image'].sample(n=50)],
    max_images=5,
    img_width=1000
)

---
---
## Create a new detection project

In this project, Tolokers select image areas that contain faces.

The first step is to configure how Tolokers will see the tasks:
- Write instructions.
- Define the input and output formats.

**Note:** It’s important to write clear instructions with examples to make sure the Tolokers do exactly what we need them to. We also recommend checking the task interface yourself to see if there are any problems with the layout.   

In [ ]:
# How tolokers will see the task
project_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ImageAnnotationFieldV1(  # Component that selects areas in images
        tb.OutputData('result'),  # Path for writing output data
        tb.InputData('image'),  # Getter for the input image
        shapes={'rectangle': True},  # Allow to select only rectangular areas #polygons
        validation=tb.RequiredConditionV1(hint='Please select an area'),  # At least one area should be selected
        full_height=True
    )
)

# You can write instructions and upload them from a file or enter them later in the web interface
# In case of polygon markup adjust the instruction describing the rules of putting the polygons instead of bounding boxes
prepared_instruction = open('./instructions/detection_instruction.html').read().strip()

# Set up the project
detection_project = toloka.Project(
    public_name='Outline all people faces with bounding boxes',
    public_description='Find and outline all people faces with bounding boxes.',
    public_instructions=prepared_instruction,
    # Set up the task: view, input, and output parameters
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec={'image': toloka.project.UrlSpec()},
        output_spec={'result': toloka.project.JsonSpec()},
        view_spec=project_interface,
    ),
)

Call the API to create a new project.

In [ ]:
detection_project = toloka_client.create_project(detection_project)

### Review your project and check the task interface

Visit the project page to make sure the task interface is working correctly.

To do this:
1. Follow the link you got in the output above.
2. In the project interface, click **Project actions** on the top right.
3. Click **Preview** in the menu.
4. Click **Change input data**.
5. Insert an image URL into the **Image** field. Here, you can use a link to one of the images you want to label, or [this sample image](https://tlk.s3.yandex.net/dataset/faces_detection/5fc9583cbda646c7de9c6451b7f22a165b6b7d4b.jpg).
6. Click the **Instructions** button. Make sure the instructions are visible and valid.
7. Try to select multiple areas with a rectangle using **Box annotation tool**.
8. Click **Submit** and then click **View responses**.

A window with the results will appear. Check that the results are in the expected format and that the data is being entered correctly.


<table  align="center">
  <tr><td>
    <img src="./img/segmentation_results_preview.png"
         alt="Task interface"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 3.</b> What the results window might look like
  </td></tr>
</table>

**Tip:** Do a trial run with a small sample of your data. Make sure that after running a trial for the entire pipeline, you get data with the expected format and quality. 

### Add custom skills for tolokers

A [skill](https://toloka.ai/docs/guide/concepts/nav.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-12) is a characteristic of a Toloker. For example, you can record the percentage of correct responses as a skill.

In this project, we’ll create two skills:
- **Detection skill**: Shows that a Toloker has completed at least one detection task. We’ll later filter out these Tolokers from verification tasks to ensure that the people doing the verification are not the same people who performed the detection.
- **Verification skill**: How good the current Toloker’s verification results are compared to others. We’ll need this skill later when we aggregate the results of the second project. 

In [ ]:
detection_skill = next(toloka_client.get_skills(name='Area selection of people faces'), None)
if detection_skill:
    print('Detection skill already exists')
else:
    detection_skill = toloka_client.create_skill(
        name='Area selection of people faces',
        hidden=True,
        public_requester_description={'EN': 'Toloker is annotating people faces'},
    )

verification_skill = next(toloka_client.get_skills(name='People faces detection verification'), None)
if verification_skill:
    print('Verification skill already exists')
else:
    verification_skill = toloka_client.create_skill(
        name='People faces detection verification',
        hidden=True,
        public_requester_description={'EN': 'How good a toloker is at verifying detection tasks'},
    )

### Pool creation for a detection project
A [pool](https://toloka.ai/docs/guide/concepts/pool-main.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-13) is a set of tasks sent out for Tolokers.

First, create an instance of a pool and set its basic parameters:
- Payment amount per task.
- Non-automatic acceptance of results.
- Number of tasks Tolokers will see per page.
- Toloker selection filters to control who can access this task.

In [ ]:
detection_pool = toloka.Pool(
    project_id=detection_project.id,
    private_name='Pool 1',  # Only you can see this information.
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),  # Pool will automatically close after one year
    reward_per_assignment=0.01,     # Set the minimum payment amount for one task page
    auto_accept_solutions=False,    # Only pay the toloker for completing the task,
                                    # based on the verification results of the second project.

    auto_accept_period_day=7,       # Number of days to determine if we'll pay for task completion by this toloker or not.
    assignment_max_duration_seconds=60*20,  # Give tolokers 20 minutes maximum to complete one task page.
    defaults=toloka.pool.Pool.Defaults(
        # We don't need overlapping for detection tasks, so we set it to 1
        default_overlap_for_new_task_suites=1,
        default_overlap_for_new_tasks=1,
    ),
)

# Set the number of tasks per page
detection_pool.set_mixer_config(real_tasks_count=1)
# Please note that the payment amount specified when creating the pool is the amount the toloker receives for completing one page of tasks.
# If you specify 10 tasks per page above, then reward_per_assignment will be paid for completing 10 tasks.

We’ll only show our tasks to English-speaking users because the description of the task is in English. This means that only people who speak English will be able to accept this task.  

In [ ]:
detection_pool.filter = toloka.filter.Languages.in_('EN')

**Quality control rules**

Each [quality control rule](https://toloka.ai/docs/guide/concepts/control.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-14) consists of the following:
- **Collector**: How to collect statistics and which metrics can be used in this rule.
- **Condition**: When the rule will be triggered. Under this condition, only parameters that apply to the collector can be used.
- **Action**: What to do if the condition is true.

In [ ]:
# The first rule in this project restricts pool access for tolokers who often make mistakes
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AcceptanceRate(),
    conditions=[
        # Toloker completed more than 2 tasks
        toloka.conditions.TotalAssignmentsCount > 2,
        # And more than 35% of their responses were rejected
        toloka.conditions.RejectedAssignmentsRate > 35,
    ],
    # This action tells Toloka what to do if the condition above is True
    # In our case, we'll restrict access for 15 days
    # Always leave a comment: it may be useful later on
    action=toloka.actions.RestrictionV2(
        scope='ALL_PROJECTS',
        duration=15,
        duration_unit='DAYS',
        private_comment='Tolokers often make mistakes',  # Only you will see this comment
    )
)

# The second useful rule is "Fast responses". It allows us to filter out tolokers who respond too quickly.
detection_pool.quality_control.add_action(
    # Let's monitor fast submissions for the last 5 completed task pages
    # And define ones that take less than 20 seconds as quick responses.
    collector=toloka.collectors.AssignmentSubmitTime(history_size=5, fast_submit_threshold_seconds=20),
    # If we see more than one fast response, we ban the toloker from all our projects for 10 days.
    conditions=[toloka.conditions.FastSubmittedCount > 1],
    action=toloka.actions.RestrictionV2(
        scope='ALL_PROJECTS',
        duration=10,
        duration_unit='DAYS',
        private_comment='Fast responses',  # Only you will see this comment
    )
)

# Another rule we use is for automatically updating skills
# We update the detection skill for tolokers who complete at least one page of tasks from detection pool.
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AnswerCount(),
    # If toloker completed at least one task, it sets the new skill to 1
    conditions=[toloka.conditions.AssignmentsAcceptedCount > 0],
    action=toloka.actions.SetSkill(skill_id=detection_skill.id, skill_value=1),
)

# This rule sends rejected assignments (tasks that you rejected) to other tolokers according to specified parameters.
detection_pool.quality_control.add_action(
    collector=toloka.collectors.AssignmentsAssessment(),
    # Check if a task was rejected
    conditions=[toloka.conditions.AssessmentEvent == 'REJECT'],
    # If the condition is True, add 1 to overlap and open the pool
    action=toloka.actions.ChangeOverlap(delta=1, open_pool=True),
)

print('Quality rules count:', len(detection_pool.quality_control.configs))

### Create a pool with all specified conditions

Now we call the Toloka API to create a pool in the detection project. Afterwards, you can check the pool in the web interface. You’ll see there aren’t any tasks in it. We’ll add them later.  

In [ ]:
detection_pool = toloka_client.create_pool(detection_pool)

---
---
## Create a new project for verification
In this project, Tolokers will determine whether the faces were outlined correctly or not.

This will be a standard classification project with only two classes: `OK` and `BAD`. We’ll explicitly define these labels as the output values.       


In [ ]:
# Configure task interface: how tolokers will see the task
verification_interface = toloka.project.TemplateBuilderViewSpec(
    view=tb.ListViewV1(  # List of components that should be positioned from top to bottom in the UI
        [
            tb.ImageAnnotationFieldV1(  # Image and selected areas to verify
                tb.InternalData('selection',
                                default=tb.InputData('selection')),  # Use the input field as default value to display the selected areas
                tb.InputData('image'),
                disabled=True,  # Disable adding and deleting areas
                full_height=True
            ),
            tb.RadioGroupFieldV1(  # A component for selecting one value out of several options
                tb.OutputData('result'),  # Path for writing output data
                [
                    tb.GroupFieldOption('OK', 'Yes'),
                    tb.GroupFieldOption('BAD', 'No'),
                ],
                label='Are all people faces outlined correctly?',  # Label above the options
                validation=tb.RequiredConditionV1()  # Requirement to select one of the options
            )
        ]
    ),
    plugins=[
        tb.HotkeysPluginV1( # Shortcuts for selecting options using the keyboard
            key_1=tb.SetActionV1(tb.OutputData('result'), 'OK'),
            key_2=tb.SetActionV1(tb.OutputData('result'), 'BAD')
        )
    ]
)

# You can write instructions and upload them from a file or enter them later in the web interface
# In case of polygon markup adjust the instruction describing the rules of putting the polygons instead of bounding boxes
verification_instruction = open('./instructions/verification_instruction.html').read().strip()

# Set up the project
verification_project = toloka.Project(
    public_name='Are the people faces outlined correctly?',
    public_description='Look at the image and decide whether or not the people faces are outlined correctly',
    public_instructions=verification_instruction,
    # Set up the task: view, input, and output parameters
    task_spec=toloka.project.task_spec.TaskSpec(
        input_spec={
            'image': toloka.project.UrlSpec(),
            'selection': toloka.project.JsonSpec(),
            'assignment_id': toloka.project.StringSpec(),
        },
        # Set allowed_values, we'll use smart mixing to get the results of this project
        output_spec={'result': toloka.project.StringSpec(allowed_values=['OK', 'BAD'])},
        view_spec=verification_interface,
    ),
)

Call the API to create a new project.

In [ ]:
verification_project = toloka_client.create_project(verification_project)

Examine the project in the web interface:

1. After running the code above, you’ll get a link to your project. Follow this link to check the task interface and instructions.

    **Note:** You will see almost the same interface as in the previous project, just without the ability to select areas. It’s important to make sure that the annotation results from the first project are displayed correctly in the second one.
    
    
2. Open the task **Preview** in the first project.


3. Outline the faces and click **Submit**.


4. Copy the results.


5. Now open the **Preview** of the second project.


6. Click **Change input data** and paste the annotation results in the selection field.


7. Click **Apply** and make sure the annotation displays correctly. 

### Create and set up a pool in the verification project
Now we need to add a filter for this pool. Specify Tolokers that don’t have the detection skill (we want to avoid using Tolokers who did the detection tasks). You can combine multiple conditions using the `&` and `|` operators.

**Note:** Add two quality control rules with the same collector but with different conditions and actions.    

In [ ]:
verification_pool = toloka.Pool(
    project_id=verification_project.id,
    private_name='Pool 1. People faces verification',  # Only you can see this information.
    may_contain_adult_content=False,
    will_expire=datetime.datetime.utcnow() + datetime.timedelta(days=365),  # Pool will close automatically after one year
    reward_per_assignment=0.01,  # We set the minimum payment amount for one task page
                                 # By default, auto_accept_solutions is on,
                                 # so we'll pay for all the tasks without checking results.
    assignment_max_duration_seconds=60*10,  # Give tolokers 10 minutes to complete one task page
    defaults=toloka.pool.Pool.Defaults(
        # We need an overlap to compare the tolokers among themselves,
        default_overlap_for_new_task_suites=5,
    ),
)

# We'll only show our tasks to English-speaking users because the description of the task is in English.
# We also won't allow our verification tasks to be performed by users who performed detection tasks.
verification_pool.filter = (
    (toloka.filter.Languages.in_('EN')) &
    (toloka.filter.Skill(detection_skill.id) == None)
)

# Set up quality control
# Quality is based on the majority of matching responses from tolokers who completed the same task.
verification_pool.quality_control.add_action(
    collector=toloka.collectors.MajorityVote(answer_threshold=3),
    # If a toloker has 10 or more responses
    # And the responses are correct in less than 50% of cases,
    conditions=[
        toloka.conditions.TotalAnswersCount > 9,
        toloka.conditions.CorrectAnswersRate < 50,
    ],
    # We ban the toloker from all our projects for 10 days.
    action=toloka.actions.RestrictionV2(
        scope='ALL_PROJECTS',
        duration=10,
        duration_unit='DAYS',
        private_comment=' Doesn\'t match the majority',  # Only you will see this comment
    )
)

# Set up the new skill value using MajorityVote.
# Depending on the percentage of correct responses, we increase the value of the toloker's skill.
verification_pool.quality_control.add_action(
    collector=toloka.collectors.MajorityVote(answer_threshold=3, history_size=10),
    conditions=[
        toloka.conditions.TotalAnswersCount > 2,
    ],
    action=toloka.actions.SetSkillFromOutputField(
        skill_id=verification_skill.id,
        from_field='correct_answers_rate',
    ),
)
print(f'Quality rule count:{len(verification_pool.quality_control.configs)}')

### Create a pool

In [ ]:
# Set the task count for one page
verification_pool.set_mixer_config(
    real_tasks_count=10,
    force_last_assignment=True,
)

verification_pool = toloka_client.create_pool(verification_pool)

---
---
## Add tasks to pools and run the projects
At this point, we have configured two projects, and now we can upload the real data that we want to annotate.

In [ ]:
tasks = [
    toloka.Task(input_values={'image': url}, pool_id=detection_pool.id)
    for url in dataset['image'].values
]
# Add tasks to a pool
toloka_client.create_tasks(tasks, allow_defaults=True,)

detection_pool = toloka_client.open_pool(detection_pool.id)

Visit the pool page in the web interface and make sure the number of tasks is correct and the pool is running. Some tasks may already be completed.

<table  align="center">
  <tr><td>
    <img src="./img/segmentation_pool_look.png"
         alt="Pool with tasks"  width="800">
  </td></tr>
  <tr><td align="center">
    <b>Figure 4.</b> What a running pool might look like
  </td></tr>
</table>


Tolokers work really fast, but they still need time to complete their tasks. We’ll use streaming to start the verification project without having to wait until the detection pool closes completely.

Next, [review](https://toloka.ai/docs/guide/concepts/offline-accept.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-15) the detection results in the web interface.

In [ ]:
from toloka.streaming import AssignmentsObserver, Pipeline
from toloka.streaming.storage import JSONLocalStorage

In [ ]:
# class for handling submissions in the detection pool
class DetectionSubmittedHandler:
    def __init__(self, client, verification_pool_id):
        self.client = client
        self.verification_pool_id = verification_pool_id

    # create new tasks for the verification pool
    def __call__(self, events: List[AssignmentEvent]) -> None:
        verification_tasks = [
            toloka.Task(
                pool_id=self.verification_pool_id,
                input_values={
                        'image': event.assignment.tasks[0].input_values['image'],
                        'selection': event.assignment.solutions[0].output_values['result'],
                        'assignment_id': event.assignment.id,
                }
            )
            for event in events
        ]
        self.client.create_tasks(verification_tasks, allow_defaults=True, open_pool=True)

In [ ]:
# class for handling accepted tasks in the verification pool
class VerificationDoneHandler:
    def __init__(self, client, verification_skill_id):
        self.microtasks = pd.DataFrame([], columns=['task', 'label', 'worker'])
        self.client = client
        self.verification_skill_id = verification_skill_id

    # filter out tasks that already have enough overlap and aggregate the result
    def __call__(self, events: List[AssignmentEvent]) -> None:
        # Initializing data
        microtasks = pd.concat([self.microtasks, self.as_frame(events)])
        # get user skills for aggregation
        skills = pd.Series({
            skill.user_id: skill.value
            for skill in self.client.get_user_skills(skill_id=self.verification_skill_id)
        })

        # Filtering all microtasks that have overlap of 5
        microtasks['overlap'] = microtasks.groupby('task')['task'].transform('count')
        to_aggregate = microtasks[microtasks['overlap'] >= 5]
        microtasks = microtasks[microtasks['overlap'] < 5]
        aggregated = MajorityVote(on_missing_skill='value', default_skill=0).fit_predict(to_aggregate, skills)
        # Accepting or rejecting assignments
        for assignment_id, result in aggregated.items():
            if result == 'OK':
                self.client.accept_assignment(assignment_id, 'Well done!')
            else:
                toloka_client.reject_assignment(assignment_id, 'The object wasn\'t selected or was selected incorrectly.')

        # Updating mictotasks
        self.microtasks = microtasks[['task', 'label', 'worker']]

    # get the data necessary for aggregation
    @staticmethod
    def as_frame(events: List[AssignmentEvent]) -> pd.DataFrame:
        microtasks = [
            (task.input_values['assignment_id'], solution.output_values['result'], event.assignment.user_id)
            for event in events
            for task, solution in zip(event.assignment.tasks, event.assignment.solutions)
        ]
        return pd.DataFrame(microtasks, columns=['task', 'label', 'worker'])

We’ll create a pipeline with an observer for each pool.

Depending on the number of images in the detection pool and the time of day, the whole process can take from a few minutes to almost an hour. 

In [ ]:
detection_observer = AssignmentsObserver(toloka_client, detection_pool.id)
detection_observer.on_submitted(DetectionSubmittedHandler(toloka_client, verification_pool.id))
verification_observer = AssignmentsObserver(toloka_client, verification_pool.id)
verification_observer.on_accepted(VerificationDoneHandler(toloka_client, verification_skill.id))

# Create a local directory that will store pipeline progress and logs.
# It allows to restart the pipeline without losing data in case of pause or failure.
storage_path = './storage/'
if not os.path.exists(storage_path):
    os.makedirs(storage_path)
storage = JSONLocalStorage(storage_path)

pipeline = Pipeline(storage=storage)
pipeline.register(detection_observer)
pipeline.register(verification_observer)

# Google Colab is using a global event pool,
# so in order to run our pipeline we have to apply nest_asyncio to create an inner pool
if 'google.colab' in str(get_ipython()):
    import nest_asyncio, asyncio
    nest_asyncio.apply()
    asyncio.get_event_loop().run_until_complete(pipeline.run())
else:
    await pipeline.run()

---
---
## Get the results
Now you can download all the accepted tasks from the detection pool and work with them. In this notebook, we’ll only show the detection results. The code below will display the results of the bounding boxes markup.

You can also [download](https://toloka.ai/docs/guide/concepts/result-of-eval.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-16) results as a TSV file from the web interface.  

In [ ]:
!pip install pillow # To deal with images
!pip install requests # To make HTTP requests
from PIL import Image, ImageDraw
import requests

def get_image(url, selection):
    raw_image = requests.get(url, stream=True).raw
    image = Image.open(raw_image).convert("RGBA")
    regions = Image.new('RGBA', image.size, (255,255,255,0))
    pencil = ImageDraw.Draw(regions)
    for region in selection:
        if region['shape'] != 'rectangle':
            continue
        p1_x = region['left'] * image.size[0]
        p1_y = region['top'] * image.size[1]
        p2_x = (region['left'] + region['width']) * image.size[0]
        p2_y = (region['top'] + region['height']) * image.size[1]
        pencil.rectangle((p1_x, p1_y, p2_x, p2_y), fill =(255, 30, 30, int(255*0.5)))
    image = Image.alpha_composite(image, regions)
    return image

detection_result = {}  # We'll store our result here

In [ ]:
max_images = 2
images = []

if not detection_result:

    for assignment in toloka_client.get_assignments(
        status='ACCEPTED',
        pool_id=detection_pool.id
    ):
        detection_result[assignment.tasks[0].input_values['image']] = assignment.solutions[0].output_values['result']

for i in range(max_images):
    url, selection = detection_result.popitem()
    image = get_image(url, selection)
    images.append(image)

ipyplot.plot_images(
    images,
    max_images=max_images,
    img_width=1000
)

---
---
## Summary

This project consists of the minimum number of settings that This project has the minimum number of settings that will allow you to collect annotated images for your dataset right from Jupyter Notebook.

For further experiments use the [Toloka-Kit documentation](https://toloka.ai/docs/toloka-kit/index.html/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-17) and check out other [use cases](https://github.com/Toloka/toloka-kit/tree/main/examples/?utm_source=github&utm_medium=instruction-b&utm_campaign=link-18).  
